In [1]:
import pandas as pd

df = pd.read_csv("C:\\Users\\STAR\\Downloads\\ipl-next-over-runs\\Data\\processed\\ipl_next_over_ml.csv")

In [2]:
df.head()

,match_id,innings,over,batting_team,venue,runs_in_over,wickets_in_over,runs_next_over,overs_completed,total_runs_so_far,current_run_rate,total_wickets,wickets_remaining,runs_last_3_overs,wickets_last_3_overs,over_phase
0,335982,1,0,Kolkata Knight Riders,M Chinnaswamy Stadium,3,0,18.0,1,3,3.0,0,10,3.000000,0.0,0
1,335982,1,1,Kolkata Knight Riders,M Chinnaswamy Stadium,18,0,6.0,2,21,10.5,0,10,10.500000,0.0,0
2,335982,1,2,Kolkata Knight Riders,M Chinnaswamy Stadium,6,0,23.0,3,27,9.0,0,10,9.000000,0.0,0
3,335982,1,3,Kolkata Knight Riders,M Chinnaswamy Stadium,23,0,10.0,4,50,12.5,0,10,15.666667,0.0,0
4,335982,1,4,Kolkata Knight Riders,M Chinnaswamy Stadium,10,0,1.0,5,60,12.0,0,10,13.000000,0.0,0


In [3]:
features = [
    "over",
    "runs_in_over",
    "runs_last_3_overs",
    "wickets_last_3_overs",
    "current_run_rate",
    "wickets_remaining",
    "over_phase"
]

X = df[features]
y = df["runs_next_over"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=200,
    max_depth=8,
    random_state=42
)

model.fit(X_train, y_train)
preds = model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
print("MAE:", mae)

MAE: 3.8607161026007324


In [7]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
    --------------------------------------- 1.0/72.0 MB 5.0 MB/s eta 0:00:15
   -- ------------------------------------- 3.7/72.0 MB 9.5 MB/s eta 0:00:08
   --- ------------------------------------ 6.3/72.0 MB 10.2 MB/s eta 0:00:07
   ---- ----------------------------------- 8.9/72.0 MB 10.6 MB/s eta 0:00:06
   ----- ---------------------------------- 10.7/72.0 MB 10.5 MB/s eta 0:00:06
   ------- -------------------------------- 13.1/72.0 MB 10.5 MB/s eta 0:00:06
   -------- ------------------------------- 15.5/72.0 MB 10.7 MB/s eta 0:00:06
   ---------- ----------------------------- 18.1/72.0 MB 10.9 MB/s eta 0:00:05
   ----------- ---------------------------- 20.7/72.0 MB 11.0 MB/s eta 0:00:05
   ------------ --------------------------- 23.3/72.0 MB 11.0 MB/s eta 0:00:05
   -------------- ------------------------- 25.7/72.0 MB 11.1 MB/s e

In [8]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)

print("XGB MAE:", mean_absolute_error(y_test, xgb_preds))

XGB MAE: 3.8655928534566484


In [9]:
results = {}

for phase in [0, 1, 2]:
    phase_df = df[df["over_phase"] == phase]

    X = phase_df[features]
    y = phase_df["runs_next_over"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    model = XGBRegressor(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.07,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    results[phase] = mae

results


{0: 3.958895838281976, 1: 3.625645342446023, 2: 4.371535033190081}

In [10]:
venue_rr = (
    df.groupby("venue")["runs_in_over"]
    .mean()
)

df["venue_avg_runs"] = df["venue"].map(venue_rr)


In [11]:
df["run_trend"] = (
    df.groupby(["match_id", "innings"])["runs_in_over"]
    .diff()
).fillna(0)


In [17]:
preds = model.predict(X_test)

In [18]:
test_df = X_test.copy()
test_df["actual"] = y_test.values
test_df["pred"] = preds

In [19]:
test_df["edge"] = test_df["pred"] - test_df["runs_in_over"]

In [20]:
test_df["trade"] = test_df["edge"].apply(
    lambda x: "OVER" if x >= 1.5 else ("UNDER" if x <= -1.5 else "NO_TRADE")
)

In [21]:
model.fit(X_train, y_train)
preds = model.predict(X_test)

test_df = X_test.copy()
test_df["actual"] = y_test.values
test_df["pred"] = preds


In [22]:
# Train model
model.fit(X_train, y_train)

# Predict on test set
preds = model.predict(X_test)


In [23]:
# Create dataframe with predictions
test_df = X_test.copy()
test_df["actual"] = y_test.values
test_df["pred"] = preds


In [24]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(test_df["actual"], test_df["pred"])
print("MAE:", mae)


MAE: 4.371535033190081


In [25]:
# Edge = model advantage
test_df["edge"] = test_df["pred"] - test_df["runs_in_over"]

# Trade rule (strict)
test_df["trade"] = test_df["edge"].apply(
    lambda x: "OVER" if x >= 1.5 else ("UNDER" if x <= -1.5 else "NO_TRADE")
)

# How many trades?
test_df["trade"].value_counts()


trade
OVER        633
UNDER       588
NO_TRADE    398
Name: count, dtype: int64

In [26]:
trade_df = test_df[
    (test_df["trade"] != "NO_TRADE") &
    (test_df["over_phase"] != 2) &
    (test_df["wickets_remaining"] >= 5)
]

len(trade_df)


0

In [27]:
bankroll = 10000
stake_pct = 0.02  # 2% risk per trade

for _, row in trade_df.iterrows():
    stake = bankroll * stake_pct
    
    # proxy win condition (temporary)
    win = abs(row["actual"] - row["pred"]) < 2

    if win:
        bankroll += stake * 0.9
    else:
        bankroll -= stake

bankroll


10000